# Space-time partial differential equation: Study of the diffusive equation (implicit methods)

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import nm_lib as nm 
import importlib
importlib.reload(nm)

<module 'nm_lib' from '/Users/juanms/Numerical_sim_course/students_2023/master/Christophe_Kristian_Blomsen/nm_lib/nm_lib/__init__.py'>

Let's consider now the viscous term in Burger's equation: 

$$\frac{\partial u}{\partial t} = \nu \frac{\partial^2 u}{\partial x^2}, \tag{1}$$

## 1- Apply an explicit method. 

What would be the CFL condition for a viscous term where $\nu$ is either a constant or an array that depends on $x$. We would like to solve equation (1) numerically for $x  [x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, periodic boundary conditions and with the initial condition:

$$u(x,t=t_0) = A\exp(-(x-x_0)^2/W^2)   \tag{2}$$

with $A=0.3$, $W=0.1$, and $x_0=0$. __Suggestion__: Apply the first derivative upwind and the second downwind. Apply Von Newman analysis. Is it stable? What is the time-step dependence with $\Delta x$? 

How many steps are needed to reach a $t=1.8$ for $nump=128$? And $256$? 

In [2]:
def u(x, x0=0, A=0.3, W=0.1):
    ans = A*np.exp(-((x - x0)/W)**2)
    return ans

def spatial_domain(nump, x0=-2/6, xf=2.6):
    ans = np.arange(nump)/(nump - 1) * (xf - x0) + x0
    return ans

Using the suggestion we get that
$$
\frac{u^{n+1}_{j} - u^{n}_{j}}{\Delta t} = \nu \frac{u_{j+1}^{n} - 2u_{j}^{n} + u_{j-1}^{n}}{\Delta x^2}
$$

Now using the Von Newmann analysis for the equation above.

$$
\begin{align}
    u_{j}^{n} &= \xi^{n} e^{ijk\Delta x}\\
    u_{j}^{n+1} &= \xi^{n+1} e^{ijk\Delta x}\\
    u_{j+1}^n &= \xi^{n} e^{i(j+1)k\Delta x} \\
    u_{j-1}^n &= \xi^{n} e^{i(j-1)k\Delta x} \\
\end{align}
$$
Using $\frac{\nu\Delta t}{\Delta x^2}=\chi$, and $\frac{k\Delta x}{2}=\alpha$
We get the following



$$
\begin{align}
e^{ijk\Delta x}\xi^{n}\frac{\xi - 1}{\Delta t} &= \nu\xi^{n}e^{ijk\Delta x} \frac{e^{ik\Delta x} -2 + e^{-ik\Delta x}}{\Delta x^2}\\
\xi &= \chi\left(e^{ik\Delta x} - 2 + e^{-ik\Delta x}\right) +1\\
\xi &= \chi\left(\cos(k\Delta x) + i\sin(k\Delta x) + \cos(k\Delta x) - i\sin(k\Delta x) -2 \right) +1\\
\xi &= 2\chi\left(\cos(k\Delta x) - 1\right) + 1\\
\xi &= 1 - 4\chi\sin^2\left(\alpha\right)\\
&\Downarrow\nonumber\\
\left|\xi\right|^2 &= 1 - 8\chi\sin^2\left(\alpha\right) + 16\chi^2 \sin^4(\alpha)
\end{align}
$$

Applying the Von Neumann stability analysis criterion
$$
\begin{align*}
\left|\xi\right|^2 &\leq 1\\
16\chi^2\sin^4(\alpha) &\leq 8\chi\sin^2(\alpha)\\
2\chi\sin^2(\alpha) &\leq 1 
\end{align*}
$$
Inserting for $\chi = \frac{\nu\Delta t}{\Delta x^2}$ and using the fact that this should apply for all values of $k$, so can just take the maximum value of $\sin$ which is 1
$$
\begin{align*}
2\frac{\nu\Delta t}{\Delta x^2} &\leq 1\\
\Delta t &\leq \frac{\Delta x^2}{2\nu}
\end{align*}
$$
Which is then is the time step dependence on $\Delta x$ for it to be stable. To reach $t=1.8$, we will then use the maximum value that $\Delta t$ value


In [3]:
nu = 1
nump_list = [128, 256]
for nump in nump_list:
    xx = spatial_domain(nump)

    dx = xx[1] - xx[0] 
    dt = dx**2/(2*nu)
    time_steps = (1.8/(np.min(dt)*np.abs(nu)))

    print(f'The amount of timesteps needed with {nump} is: {time_steps:.2f}')


The amount of timesteps needed with 128 is: 6748.19
The amount of timesteps needed with 256 is: 27205.71


<span style="color:green"> JMS. </span>

<span style="color:blue"> OK. Note that the number of steps (with double resolution) is not double, but quadruple.</span>.

## Choose one of the following options: 

## 2- Implicit methods.

In the [wiki](https://github.com/AST-Course/AST5110/wiki/Implicit-methods), we describe some implicit or semi-explicit methods that allow relaxing the CFL constraint on diffusive terms. Consider Newton-Rapson method and repeat the previous numerical experiment. For this, you will need to implement the following   


$F_j = u^{n+1}_j - u^n_j - \nu \, (u^{n+1}_{j+1} - 2u^{n+1}_{j}+u^{n+1}_{j-1})\frac{\Delta t}{\Delta x^2}$

in `NR_f` and `step_diff_burgers` functions in `nm_lib`. 

And the Jacobian can be easily built. 

$J(j,k) = F_j'(u^{n+1}_k)$

fill in the `jacobian` function in `nm_lib`. Note that this matrix is linear with $u$. 

Test the model with [wiki](https://github.com/AST-Course/AST5110/wiki/Self-similar-solution-for-parabolic-eq) self-similar solutions. How long it takes each time step compared to the Lax-method? Use `time.time` library. Do it for nump=256, nt=30 and dt = 0.1. In order to test the simulation, use `curve_fit` from `scipy.optimize`. 

__hint__ consider to use a good initial guess (`p0`) in and `bnd_limits` to facilitate the fitting wiht `curve_fit`. What happens to the solution when increasing dt? How much can be improved in limiting the tolerance?

----------------------------------------------

Let's consider a non-linear function where $\nu$ depends on $u$. To keep it simple, solve the following: 

$\frac{\partial u}{\partial t} = u \frac{\partial^2 u}{\partial x^2}$

where $\nu_0$ is a constant and the same initial conditions as the previous exercise (fill in `Newton_Raphson_u`, `jacobian_u` and `NR_f_u`. Consider an error limit of $10^{-4}$ and compare the previous exercise (with the same error limit). How many iterations needs now the method to converge to the right solution? Why? Increase `ncount` to 1000. 

## 3- Semi-explicit methods. 

__a)__ Super-time-stepping (STS) schemes work for parabolic terms. STS is an API method that performs a subset of "unstable" intermediate steps, but the sum of all the steps is stable. Visualize how `taui_sts` varies with $nu$ and $niter$. Compare the solution with the analytical one for the final and intermediate STS steps. For the full STS steps, how improves the solution with $nu$? and $niter$? Is there a relation between the error and these two parameters, $nu$, and $niter$? For which $niter$ and $nu$ the method provides larger steps than an ordinary explicit. For this exercise, fill in `evol_sts`, and `taui_sts`. 